This scrip is made to create a df_lookup that will be used as index to complete the missing values in the df (based on event.csv table of bigquery-public-data.thelook_ecommerce) to create a df_final to be saved as event_clean.csv 

In [4]:
#library import
import re
import geopy
import pgeocode
import requests
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
#upload and dataset visualization
df = pd.read_csv("events_cleaned.csv")
df.head()

,event_id,user_id,sequence_number,created_at,ip_address,city,state,postal_code,browser,traffic_source,event_type
0,1731488,NaN,3,2020-08-02 07:15:00,158.47.215.35,São Paulo,São Paulo,02675-031,Firefox,Email,cancel
1,1824322,NaN,3,2023-11-05 12:22:00,201.67.237.55,São Paulo,São Paulo,02675-031,Chrome,Email,cancel
2,2133186,NaN,3,2024-03-10 10:09:00,181.108.42.134,São Paulo,São Paulo,02675-031,Chrome,Facebook,cancel
3,1758184,NaN,3,2019-08-04 09:22:00,194.226.235.193,São Paulo,São Paulo,02675-031,Chrome,Email,cancel
4,1366193,NaN,3,2019-05-06 01:15:00,96.121.76.158,São Paulo,São Paulo,02675-031,Chrome,Email,cancel


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2420973 entries, 0 to 2420972
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   event_id         int64  
 1   user_id          float64
 2   sequence_number  int64  
 3   created_at       object 
 4   ip_address       object 
 5   city             object 
 6   state            object 
 7   postal_code      object 
 8   browser          object 
 9   traffic_source   object 
 10  event_type       object 
dtypes: float64(1), int64(2), object(8)
memory usage: 203.2+ MB


In [7]:
# puntual data error fetching and cleaning
df.loc[(df['postal_code'] == '30045') & (df['state'] == 'Shanxi'), ['city', 'postal_code']] = ['Xiqing', '030045']
df.loc[(df['postal_code'] == '300384') & (df['state'] == 'Tianjin'), ['city', 'postal_code']] = ['Xiqing', '300384']
df.loc[(df['postal_code'] == '29142') & (df['state'] == 'Espírito Santo'), ['city', 'postal_code']] = ['Cariacica', '29142-000']

In [8]:
#tuplas compose dictionary creation for problematic and dupolicate postal_code val
fix_map = {
    ("29140", "Bretagne"): "Rosporden",
    ("29140", "Andalucía"): "Málaga",
    ("38300", "Canarias"): "La Orotava",
    ("38300", "Isère"): "Bourgoin-Jallieu",
    ("13090", "Bouches-du-Rhône"): "Aix-en-Provence",
    ("13090", "New York"): "Liverpool",
    ("3581", "Flanders"): "Beringen",
    ("3581", "Comunidad Valenciana"): "L'Alfàs del Pi",
    ("46988", "Comunidad Valenciana"): "Paterna",
    ("28946", "Comunidad de Madrid"): "Fuenlabrada", 
    ("300384", "Tianjin"): "Xiqing",
    ("33129", "Nordrhein-Westfalen"): "Delbrück",
}

In [9]:
#dictionary application
df["city"] = df.apply(
    lambda row: fix_map.get((row["postal_code"], row["state"]), row["city"]),
    axis=1
)

In [10]:
# NaN values for postal codes searching
missing_codes = df[df['city'].isnull()]['postal_code'].unique()
print(missing_codes) 
print(len(missing_codes))  

['25191' '30016' '30044' '30093' '47240-000' '56460-000' '641-920'
 '65725-000' '68473-000' '68540-000' '69200-000' '69435-000' '69880-000'
 '75370-000' '15174' '20112' '23188' '27537' '30045' '32060' '32826'
 '32828' '39260-000' '46394' '48044' '48475-000' '48790-000' '55730-000'
 '56180-000' '56380-000' '57360-000' '630-520' '63610-000' '65393-000'
 '65690-000' '68330-000' '68617-000' '68620-000' '68650-000' '68660-000'
 '68695-000' '68890-000' '68924-000' '69280-000' '69460-000' '69630-000'
 '69800-000' '69830-000' '72025-065' '72110-600' '76868-000' '77379'
 '78850-000' '84600-000' '20171' '22408' '28947' '30102' '33579' '34476'
 '47350-000' '48760-000' '50016' '50191' '55920-000' '641-860' '65380-000'
 '65415-000' '68300-000' '69230-000' '69250-000' '69600-000' '69735-000'
 '72016-190' '73900-000' '76980-000' '79706' '630-490' '630-500' '22556'
 '22602' '28909' '32940' '44330-000' '50197' '630-857' '631-830' '642-370'
 '65278-000' '65530-000' '68230-000' '68250-000' '69980-000' '9

In [11]:
#df_lookup dataframe creation
df_lookup = df[df['postal_code'].isin(missing_codes)][['postal_code', 'state']].drop_duplicates()

In [12]:
print(len(df_lookup))
print(df_lookup)

132
       postal_code                 state
334          25191              Cataluña
414          30016               Georgia
425          30044               Georgia
431          30093               Georgia
732      47240-000                 Bahia
889      56460-000            Pernambuco
985        641-920      Gyeongsangnam-do
1013     65725-000              Maranhão
1040     68473-000                  Pará
1048     68540-000                  Pará
1059     69200-000              Amazonas
1061     69435-000              Amazonas
1063     69880-000              Amazonas
1132     75370-000                 Goiás
2111         15174               Galicia
2393         20112              Virginia
2746         23188              Virginia
2997         27537        North Carolina
3219         30045               Georgia
3462         32060               Florida
3550         32826               Florida
3551         32828               Florida
4024     39260-000          Minas Gerais
4664        

In [13]:
#looking for city by postal code and state using OpenStreetMap API function definition
def get_city_osm(postal_code, state=None):
    url = "https://nominatim.openstreetmap.org/search"
    headers = {"User-Agent": "AlessandroPostalLookup/1.0"}

    params = {
        "postalcode": postal_code,
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }

    try:
        r = requests.get(url, params=params, headers=headers, timeout=10)
        if r.status_code == 200 and len(r.json()) > 0:
            addr = r.json()[0]["address"]
            return (
                addr.get("city") or
                addr.get("town") or
                addr.get("village") or
                addr.get("municipality")
            )
    except:
        pass

    
    if state:
        params["state"] = state
        try:
            r = requests.get(url, params=params, headers=headers, timeout=10)
            if r.status_code == 200 and len(r.json()) > 0:
                addr = r.json()[0]["address"]
                return (
                    addr.get("city") or
                    addr.get("town") or
                    addr.get("village") or
                    addr.get("municipality")
                )
        except:
            pass

    return None

In [14]:
#looking for city by postal code and state using OpenStreetMap API 
results = []

missing_rows = df[df['city'].isnull()][['postal_code', 'state']].drop_duplicates()

for idx, row in missing_rows.iterrows():
    pc = row['postal_code']
    state = row['state']

    city = get_city_osm(pc, state)

    results.append({
        "postal_code": pc,
        "city": city
    })

    print(f"{pc} ({state}) → {city}")

    time.sleep(1) #respecting OSM rate limit

25191 (Cataluña) → Lleida
30016 (Georgia) → Tebesbest ⵜⴰⴱⴻⵙⴱⴻⵙⵜ تبسبست
30044 (Georgia) → 세종특별자치시
30093 (Georgia) → None
47240-000 (Bahia) → None
56460-000 (Pernambuco) → Petrolândia
641-920 (Gyeongsangnam-do) → None
65725-000 (Maranhão) → Pedreiras
68473-000 (Pará) → Novo Repartimento
68540-000 (Pará) → Conceição do Araguaia
69200-000 (Amazonas) → Borba
69435-000 (Amazonas) → None
69880-000 (Amazonas) → Eirunepé
75370-000 (Goiás) → Goianira
15174 (Galicia) → Culleredo
20112 (Virginia) → Tambor
23188 (Virginia) → None
27537 (North Carolina) → Henderson
30045 (Georgia) → Берездівська сільська громада
32060 (Florida) → None
32826 (Florida) → Cartelle
32828 (Florida) → Celanova
39260-000 (Minas Gerais) → Várzea da Palma
46394 (Comunidad Valenciana) → Riba-roja de Túria
48044 (Michigan) → 부산광역시
48475-000 (Bahia) → Itapicuru
48790-000 (Bahia) → Tucano
55730-000 (Pernambuco) → Bom Jardim
56180-000 (Pernambuco) → Cabrobó
56380-000 (Pernambuco) → Santa Maria da Boa Vista
57360-000 (Alagoas) → G

In [15]:
#df_results dataframe creation
df_results = pd.DataFrame(results)
print(df_results)

    postal_code                           city
0         25191                         Lleida
1         30016     Tebesbest ⵜⴰⴱⴻⵙⴱⴻⵙⵜ تبسبست
2         30044                        세종특별자치시
3         30093                           None
4     47240-000                           None
5     56460-000                    Petrolândia
6       641-920                           None
7     65725-000                      Pedreiras
8     68473-000              Novo Repartimento
9     68540-000          Conceição do Araguaia
10    69200-000                          Borba
11    69435-000                           None
12    69880-000                       Eirunepé
13    75370-000                       Goianira
14        15174                      Culleredo
15        20112                         Tambor
16        23188                           None
17        27537                      Henderson
18        30045  Берездівська сільська громада
19        32060                           None
20        328

In [16]:
#df_lookup data enrichment
df_lookup = df_lookup.merge(
    df_results,
    on="postal_code",
    how="left",
)

In [17]:
print(len(df_lookup))
print(df_lookup)

132
    postal_code                 state                           city
0         25191              Cataluña                         Lleida
1         30016               Georgia     Tebesbest ⵜⴰⴱⴻⵙⴱⴻⵙⵜ تبسبست
2         30044               Georgia                        세종특별자치시
3         30093               Georgia                           None
4     47240-000                 Bahia                           None
5     56460-000            Pernambuco                    Petrolândia
6       641-920      Gyeongsangnam-do                           None
7     65725-000              Maranhão                      Pedreiras
8     68473-000                  Pará              Novo Repartimento
9     68540-000                  Pará          Conceição do Araguaia
10    69200-000              Amazonas                          Borba
11    69435-000              Amazonas                           None
12    69880-000              Amazonas                       Eirunepé
13    75370-000               

In [18]:
# puntual data error fetching and cleaning
df_lookup.loc[df_lookup['state'] == 'Gyeongsangnam-do', 'city'] = 'Changwon'

In [19]:
#tuplas compose dictionary creation for wrong city values
fix_map = {
    ("30044", "Georgia"): "Lawrenceville",
    ("30045", "Georgia"): "Lawrenceville",
    ("30016", "Georgia"): "Covington",
    ("30102", "Georgia"): "Acworth",
    ("30028", "Georgia"): "Cumming",
    ("20171", "Virginia"): "Herndon",
    ("22602", "Virginia"): "Winchester",
    ("22556", "Virginia"): "Stafford",
    ("23236", "Virginia"): "Richmond",
    ("22408", "Virginia"): "Fredericksburg",
    ("20112", "Virginia"): "Manassas",
    ("32828", "Florida"): "Orlando",
    ("32826", "Florida"): "Orlando",
    ("32092", "Florida"): "Saint Augustine",
    ("32940", "Florida"): "Melbourne",
    ("33579", "Florida"): "Riverview",
    ("12065", "New York"): "Clifton Park",
    ("17015", "Pennsylvania"): "Carlisle",
    ("48044", "Michigan"): "Macomb",
    ("79706", "Texas"): "Midland",
    ("68617-000", "Pará"): "Cachoeira do Piriá",
    ("68330-000", "Pará"): "Porto de Moz",
    ("69280-000", "Amazonas"): "Manicoré",
    ("69435-000", "Amazonas"): "Manaquiri",
    ("62150-000", "Ceará"): "Santana do Acaraú",
    ("47240-000", "Bahia"): "Pilão Arcado",
}

In [20]:
#city correction
df_lookup["city"] = df_lookup.apply(
    lambda row: fix_map.get((row["postal_code"], row["state"]), row["city"]),
    axis=1
)

In [21]:
print(df_lookup)

    postal_code                 state                        city
0         25191              Cataluña                      Lleida
1         30016               Georgia                   Covington
2         30044               Georgia               Lawrenceville
3         30093               Georgia                        None
4     47240-000                 Bahia                Pilão Arcado
5     56460-000            Pernambuco                 Petrolândia
6       641-920      Gyeongsangnam-do                    Changwon
7     65725-000              Maranhão                   Pedreiras
8     68473-000                  Pará           Novo Repartimento
9     68540-000                  Pará       Conceição do Araguaia
10    69200-000              Amazonas                       Borba
11    69435-000              Amazonas                   Manaquiri
12    69880-000              Amazonas                    Eirunepé
13    75370-000                 Goiás                    Goianira
14        

In [22]:
#looking for missing city values
missing_info = df_lookup[df_lookup['city'].isnull()].copy()
print(f"{missing_info}\nLength: {len(missing_info)}") 

    postal_code         state  city
3         30093       Georgia  None
16        23188      Virginia  None
19        32060       Florida  None
51        77379         Texas  None
59        34476       Florida  None
69    69230-000      Amazonas  None
99        34482       Florida  None
101       77389         Texas  None
105       70706     Louisiana  None
106       22407      Virginia  None
107       23693      Virginia  None
112       33437       Florida  None
113       22406      Virginia  None
118       88201    New Mexico  None
122       19803      Delaware  None
127       18302  Pennsylvania  None
Length: 16


In [23]:
#tuplas compose dictionary creation for missing values
fix_map = {
    ("70706", "Louisiana"): "Denham Springs",
    ("77379", "Texas"): "Spring",
    ("77389", "Texas"): "Spring",
    ("23188", "Virginia"): "Williamsburg",
    ("23693", "Virginia"): "Yorktown",
    ("22407", "Virginia"): "Fredericksburg",
    ("22406", "Virginia"): "Fredericksburg",
    ("88201", "New Mexico"): "Roswell",
    ("19803", "Delaware"): "Wilmington",
    ("30093", "Georgia"): "Norcross",
    ("32060", "Florida"): "Live Oak",
    ("34476", "Florida"): "Ocala",
    ("34482", "Florida"): "Ocala",
    ("33437", "Florida"): "Boynton Beach",
    ("18302", "Pennsylvania"): "East Stroudsburg",
    ("69230-000", "Amazonas"): "Nova Olinda do Norte",
    ("46394", "Comunidad Valenciana"): "Ribarroja del Turia",
}

In [24]:
#df_lookup data enrichmen
df_lookup["city"] = df_lookup.apply(
    lambda row: fix_map.get((row["postal_code"], row["state"]), row["city"]),
    axis=1
)

In [25]:
#looking for missing values
missing_info2 = df_lookup[df_lookup['city'].isnull()].copy()
print(missing_info2)

Empty DataFrame
Columns: [postal_code, state, city]
Index: []


In [26]:
#df_final creation by merging df with df_lookup 
df_final = df.merge(
    df_lookup[['postal_code', 'city']],
    on='postal_code',
    how='left',
    suffixes=('', '_lookup')
)

df_final['city'] = df_final['city'].fillna(df_final['city_lookup'])

df_final = df_final.drop(columns=['city_lookup'])

In [27]:
print(df_final.head(10))

   event_id  user_id  sequence_number           created_at       ip_address  \
0   1731488      NaN                3  2020-08-02 07:15:00    158.47.215.35   
1   1824322      NaN                3  2023-11-05 12:22:00    201.67.237.55   
2   2133186      NaN                3  2024-03-10 10:09:00   181.108.42.134   
3   1758184      NaN                3  2019-08-04 09:22:00  194.226.235.193   
4   1366193      NaN                3  2019-05-06 01:15:00    96.121.76.158   
5   1447159      NaN                3  2022-03-25 11:42:00     76.87.229.63   
6   2195195      NaN                3  2019-02-01 08:59:00      3.151.107.2   
7   1697243      NaN                3  2022-06-12 05:24:00    17.251.215.20   
8   1478891      NaN                3  2021-06-12 08:06:00  173.131.161.162   
9   2310222      NaN                3  2020-06-02 14:36:00    122.136.53.69   

        city      state postal_code  browser traffic_source event_type  
0  São Paulo  São Paulo   02675-031  Firefox          Ema

In [28]:
#looking for missing values
missing_info3 = df_final[df_final['city'].isnull()]
print(missing_info3)

Empty DataFrame
Columns: [event_id, user_id, sequence_number, created_at, ip_address, city, state, postal_code, browser, traffic_source, event_type]
Index: []


In [ ]:
#save the final df to events_clean.csv
df_final.to_csv("events_cleaned2.csv", index=False)